In [ ]:
# Define the first cleaning function again, as the execution environment is reset between turns.
import pandas as pd

def remove_page_duplicates(data: pd.DataFrame, target_column: str = 'user_journey') -> pd.DataFrame:
    """
    Removes sequences of sequentially repeating pages in the user journey strings.

    Args:
        data: The dataframe containing all the data.
        target_column: The name of the column containing the user journey strings.

    Returns:
        A new dataframe with the cleaned-up journey strings.
    """
    # Create a copy to avoid modifying the original dataframe
    new_df = data.copy()

    def clean_journey(journey: str) -> str:
        if pd.isna(journey):
            return journey

        # Split the journey into pages
        pages = journey.split('-')
        
        # Initialize a list for the cleaned journey
        cleaned_pages = []
        
        # Iterate through the pages and keep only if it's different from the last added page
        for page in pages:
            if not cleaned_pages or page != cleaned_pages[-1]:
                cleaned_pages.append(page)
        
        # Join the pages back into a single string
        return '-'.join(cleaned_pages)

    # Apply the cleaning function to the target column
    new_df[target_column] = new_df[target_column].apply(clean_journey)
    
    return new_df

# --- Demo Data and Testing ---

# Create demo data to test the function
demo_data = pd.DataFrame({
    'user_id': [1, 2, 3, 4],
    'session_id': [101, 102, 103, 104],
    'user_journey': [
        "Homepage-Pricing-Homepage",              # Should remain unchanged
        "Homepage-Homepage-Homepage-Pricing",     # Should become "Homepage-Pricing"
        "Checkout-Checkout-Review-Review-Final",  # Should become "Checkout-Review-Final"
        "Login-Login-Homepage-Pricing-Login"      # Should become "Login-Homepage-Pricing-Login"
    ]
})

print("Original Demo Data:")
print(demo_data)
# Original Demo Data:
#    user_id  session_id                           user_journey
# 0        1         101              Homepage-Pricing-Homepage
# 1        2         102     Homepage-Homepage-Homepage-Pricing
# 2        3         103  Checkout-Checkout-Review-Review-Final
# 3        4         104     Login-Login-Homepage-Pricing-Login

# Call the function on the demo data
cleaned_demo_data = remove_page_duplicates(demo_data)

print("\nCleaned Demo Data:")
print(cleaned_demo_data)
# Cleaned Demo Data:
#    user_id  session_id                  user_journey
# 0        1         101     Homepage-Pricing-Homepage
# 1        2         102              Homepage-Pricing
# 2        3         103         Checkout-Review-Final
# 3        4         104  Login-Homepage-Pricing-Login

In [ ]:
import pandas as pd
import re

# Define the first cleaning function
def remove_page_duplicates(data: pd.DataFrame, target_column: str = 'user_journey') -> pd.DataFrame:
    # ... (function definition is the same)
    new_df = data.copy()
    
    def clean_journey(journey: str) -> str:
        if pd.isna(journey):
            return journey
        pages = journey.split('-')
        cleaned_pages = []
        for page in pages:
            if not cleaned_pages or page != cleaned_pages[-1]:
                cleaned_pages.append(page)
        return '-'.join(cleaned_pages)

    new_df[target_column] = new_df[target_column].apply(clean_journey)
    return new_df

# Load the data from the CSV file
df_raw = pd.read_csv("user_journey_raw.csv")

# Create a subset of the first 100 rows for demonstration
df_raw_100 = df_raw.head(100)

# Print the 100 rows of the RAW DataFrame
print("--- 100 Rows of the RAW DataFrame ---")
print(df_raw_100.to_string())

# Apply the cleaning function ONLY to the 100-row subset
df_cleaned_100 = remove_page_duplicates(df_raw_100)

# Print the 100 rows of the cleaned DataFrame
print("\n--- 100 Rows of the CLEANED DataFrame (only the subset was cleaned) ---")
print(df_cleaned_100.to_string())

In [ ]:
import pandas as pd

# Increase the maximum column width and set display options to ensure full string printing
""" pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 10) """

# Load the data from the CSV file
df_raw = pd.read_csv("user_journey_raw.csv")

def group_by(data: pd.DataFrame, group_column: str = 'user_id', target_column: str = 'user_journey', sessions = 'All', count_from: str = 'last') -> pd.DataFrame:
    """
    Groups user journey strings for a given user ID, optionally restricting the number of sessions.
    
    *Fixes deprecation warning by using .head() and .tail() instead of .apply() for session slicing.*
    """
    new_df = data.copy()

    # 1. Sort by group_column and session_id to ensure chronological order within each user
    if 'session_id' in new_df.columns:
        new_df = new_df.sort_values(by=[group_column, 'session_id'])
    else:
        new_df = new_df.sort_values(by=[group_column])

    # 2. Select the specified number of sessions using head/tail
    if isinstance(sessions, int) and sessions > 0:
        grouped = new_df.groupby(group_column, group_keys=False)

        if count_from == 'first':
            new_df = grouped.head(sessions)
        elif count_from == 'last':
            new_df = grouped.tail(sessions)

    # 3. Group and aggregate the journey strings, joining them with a hyphen
    grouped_df = new_df.groupby(group_column).agg(
        {target_column: lambda x: '-'.join(x.astype(str))}
    ).reset_index()

    return grouped_df

    # Print the original demo data (pre-grouped sessions)
print("--- Original Demo Data (Sessions per User) ---")
print(df_raw[['user_id', 'session_id', 'user_journey']].to_string())
# Output (partial, showing User ID 1516 and 3395 for context):
#     user_id  session_id                                                  user_journey
# 0      1516     2980231                                         Homepage-Log in-Other
# 1      1516     2980248                                          Other-Sign up-Log in
# 2      1516     2992252                                                        Log in
# 3      1516     3070491                                               Homepage-Log in
# 4      1516     3709807                                                        Log in
# 5      1516     3723132                                                      Checkout
# 6      1516     3723365                                                      Checkout
# 7      1516     3723382                                                      Checkout
# 8      1516     3723427                                                      Checkout
# 9      1516     3723483                                                        Coupon
# 10     1516     3723508                                                      Checkout
# 11     1516     3724778                                                      Checkout
# 12     1516     3726160                                                      Checkout
# 13     3395     1415870                                                         Other
# 14     3395     3645805                                        Pricing-Sign up-Log in
# 15     3395     3657408                                              Homepage-Pricing
# 16     3395     3712148                                              Pricing-Checkout
# 17     3395     3713857                                                      Checkout
# ... (and so on for other users)

# Print the session count per test user BEFORE grouping
print("\n--- Session Count Per User (for test users) ---")
print(df_raw.groupby('user_id')['session_id'].count())
# Output:
# user_id
# 1516     13
# 3395      5
# 10107    16
# 11145    11
# 12400     4
# Name: session_id, dtype: int64


# 1. Test 'All' sessions (Default)
df_grouped_all = group_by(df_raw, sessions='All')
print("\n--- Test 1: Grouping 'All' Sessions ")
print(df_grouped_all)
# Output (User 1516):
#    user_id                                                                                     user_journey
# 0     1516  Homepage-Log in-Other-Other-Sign up-Log in-Log in-Homepage-Log in-Log in-Checkout-Checkout-Checkout-Checkout-Coupon-Checkout-Checkout-Checkout
# ...

# 2. Test grouping by the 'last' 3 sessions
df_grouped_last_3 = group_by(df_raw, sessions=3, count_from='last')
print("\n--- Test 2: Grouping the 'Last' 3 Sessions ")
print(df_grouped_last_3)
# Output (User 1516):
#    user_id                 user_journey
# 0     1516  Checkout-Checkout-Checkout
# ...

# 3. Test grouping by the 'first' 2 sessions
df_grouped_first_2 = group_by(df_raw, sessions=2, count_from='first')
print("\n--- Test 3: Grouping the 'First' 2 Sessions ---")
print(df_grouped_first_2)
# Output (User 1516):
#    user_id                    user_journey
# 0     1516  Homepage-Log in-Other-Other-Sign up-Log in
    # ...

In [5]:
import pandas as pd

# Set options to display full journey strings (optional, but good practice)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 10)

# --- Function 1: Remove Sequential Page Duplicates ---
def remove_page_duplicates(data: pd.DataFrame, target_column: str = 'user_journey') -> pd.DataFrame:
    """Removes sequences of sequentially repeating pages in the user journey strings."""
    new_df = data.copy()
    def clean_journey(journey: str) -> str:
        if pd.isna(journey):
            return journey
        pages = journey.split('-')
        cleaned_pages = []
        for page in pages:
            if not cleaned_pages or page != cleaned_pages[-1]:
                cleaned_pages.append(page)
        return '-'.join(cleaned_pages)
    new_df[target_column] = new_df[target_column].apply(clean_journey)
    return new_df

# --- Function 2: Group User Journeys by ID and Session Count (MODIFIED for column order) ---
def group_by(data: pd.DataFrame, group_column: str = 'user_id', target_column: str = 'user_journey', sessions = 'All', count_from: str = 'last') -> pd.DataFrame:
    """
    Groups user journey strings for a given user ID, optionally restricting the number of sessions.
    Preserves 'subscription_type' and ensures it is the second column (index 1).
    """
    new_df = data.copy()
    if 'session_id' in new_df.columns:
        new_df = new_df.sort_values(by=[group_column, 'session_id'])
    else:
        new_df = new_df.sort_values(by=[group_column])

    if isinstance(sessions, int) and sessions > 0:
        grouped = new_df.groupby(group_column, group_keys=False)
        if count_from == 'first':
            new_df = grouped.head(sessions)
        elif count_from == 'last':
            new_df = grouped.tail(sessions)

    # MODIFICATION: Explicitly define the aggregation dictionary keys in the desired output order.
    agg_dict = {}
    if 'subscription_type' in new_df.columns:
        # 1. Place 'subscription_type' first in the aggregation dict (it will be column 2 after user_id)
        agg_dict['subscription_type'] = 'first'
        
    # 2. Place 'user_journey' second in the aggregation dict (it will be column 3)
    agg_dict[target_column] = lambda x: '-'.join(x.astype(str))
        
    grouped_df = new_df.groupby(group_column).agg(agg_dict).reset_index()
    
    # Final check and explicit reordering of columns to ensure user_id, subscription_type, user_journey
    # Although agg typically orders by dict keys, explicit selection is safest.
    final_cols = [group_column]
    if 'subscription_type' in grouped_df.columns:
        final_cols.append('subscription_type')
    final_cols.append(target_column)

    # Return the DataFrame with the correct column order
    return grouped_df[final_cols]

# --- Function 3: Remove Specific Pages from Journey String (No Change) ---
def remove_pages(data: pd.DataFrame, pages: list, target_column: str = 'user_journey') -> pd.DataFrame:
    """Removes specific pages from the user journey strings in the target column."""
    new_df = data.copy()
    pages_to_remove = set(pages)
    def filter_journey(journey: str) -> str:
        if pd.isna(journey) or not journey:
            return journey
        filtered_pages = [page for page in journey.split('-') if page not in pages_to_remove]
        return '-'.join(filtered_pages)
    new_df[target_column] = new_df[target_column].apply(filter_journey)
    return new_df

# --- FULL PIPELINE EXECUTION (Order: F2 -> F3 -> F1) ---

# Load the raw data
df_raw = pd.read_csv("user_journey_raw.csv")

# Step 1 (Function 2): Group all sessions, ensuring 'subscription_type' is column 2
df_step_2 = group_by(df_raw, sessions='All')

# Step 2 (Function 3): Remove specific pages (none yet, as requested)
pages_to_exclude = []
df_step_3 = remove_pages(df_step_2, pages=pages_to_exclude)

# Step 3 (Function 1): Remove sequential duplicates from the final long journey string.
df_final = remove_page_duplicates(df_step_3)

# 4. Export the final DataFrame to CSV
output_filename = 'user_journey_cleaned_reordered.csv'
df_final.to_csv(output_filename, index=False)

# Inspect the final DataFrame to confirm 'subscription_type' is the second column
print(f"--- Final DataFrame Head (Confirming Column Order) ---")
print(df_final.head().to_string())

print(f"\n--- Full Pipeline Reordered Execution Complete ---")
print(f"Pipeline Order: Function 2 (group_by) -> Function 3 (remove_pages) -> Function 1 (remove_page_duplicates)")
print(f"Final DataFrame has {len(df_final)} rows (one per unique user).")
print(f"Saved cleaned data to {output_filename}")

# Reset options
pd.reset_option('display.max_colwidth')
pd.reset_option('display.width')
pd.reset_option('display.max_rows')

--- Final DataFrame Head (Confirming Column Order) ---
   user_id subscription_type                                                                                                                                                                                                                     user_journey
0     1516            Annual                                                                                                                                                    Homepage-Log in-Other-Sign up-Log in-Homepage-Log in-Checkout-Coupon-Checkout
1     3395            Annual                                                                                                                                                                           Other-Pricing-Sign up-Log in-Homepage-Pricing-Checkout
2    10107            Annual  Homepage-Career tracks-Homepage-Career tracks-Sign up-Log in-Homepage-Resources center-Other-Homepage-Career tracks-Courses-Career tracks-Courses-Career 